In [ ]:
import sys; sys.path.extend(['..', '../validations/'])
import MeshFEM
import mesh, elastic_sheet, energy, tensors
from tri_mesh_viewer import TriMeshViewer, PointCloudViewer, PointCloudMesh
import py_newton_optimizer, fd_validation, benchmark
from matplotlib import pyplot as plt
import meshing, time
import numpy as np
from io_redirection import suppress_stdout as so
from ipywidgets import interact, widgets
import sim_utils, sheet_convergence

# Flap made of two equilateral triangles with dihedral angle `theta`
def flapPts(theta, heightScale=1.0, base=1.0):
    h = heightScale * np.sqrt(1 - 0.5**2)
    alpha = theta / 2 - np.pi / 2
    c, s = np.cos(alpha), np.sin(alpha)
    return [[0, 0, -0.5 * base], [0, 0, 0.5 * base], [h * c, h * s, 0], [-h * c, h * s, 0]]
    
def flapTris(theta):
    V = flapPts(theta)
    F = [[0, 1, 2], [1, 0, 3]]
    return V, F

In [ ]:
showNormals = True
thickness = 1.0

In [ ]:
#es = elastic_sheet.ElasticSheet(mesh.Mesh(*flapTris(np.pi), embeddingDimension=3), energy.StVenantKirchhoffCBased(tensors.ElasticityTensor2D(200, 0.3)))
creaseEdges = [[0, 1]]
es = elastic_sheet.ElasticSheet(mesh.Mesh(*flapTris(np.pi), embeddingDimension=3), energy.NeoHookeanYoungPoisson(2, 200, 0.3), creaseEdges)
es.thickness = thickness
view = TriMeshViewer(es, wireframe=True, width=800, height=600)
normalView = PointCloudViewer(es.edgeMidpoints(), vectorField=es.midedgeNormals(), superView=view) if showNormals else None

out = widgets.Output()
out.layout.height = '40px'

In [ ]:
def sheetUpdate(theta, height, base, creaseAngle =0.0):
    es.setDeformedPositions(flapPts(theta, heightScale=height, base=base))
    es.setCreaseAngles([creaseAngle])
    es.initializeMidedgeNormals()

def updateFlap(theta = np.pi, height=1.0, base=1.0, creaseAngle=0.0):
    sheetUpdate(theta, height, base, creaseAngle)
    if showNormals: normalView.update(preserveExisting=False, mesh=PointCloudMesh(es.edgeMidpoints()), vectorField=es.midedgeNormals())
    view.update(vectorField=-es.gradient()[0:es.thetaOffset()].reshape((-1, 3)))
    out.clear_output()
    with out:
        print('Membrane energy: ', es.energy(es.EnergyType.Membrane))
        print('Bending  energy: ', es.energy(es.EnergyType.Bending))
        print('Total    energy: ', es.energy())
    #view.update()

In [ ]:
sheetUpdate(np.pi - 1.57, 1.0, 1.0, 1.57)
es.energy()

In [ ]:
view.setDarkMode(False)
display(view.show())
display(out)

i = interact(updateFlap, theta=(0.0, 2 * np.pi, 0.01), height=(0.0, 2.0, 0.01), base=(0.0, 2.0, 0.01), creaseAngle=(-np.pi, np.pi, 0.01));
def reset(sender):
    i.widget.children[0].value = np.pi
    i.widget.children[1].value = 1.0
    i.widget.children[2].value = 1.0
    i.widget.children[3].value = 0.0
b = widgets.Button()
b.description = 'Reset'
b.on_click(reset)
display(b)

In [ ]:
scales = np.linspace(1e-4, 1.0, 1000)
energies = {}
for thetas in [12 * np.pi / 8, 13 * np.pi / 8, 13 * np.pi / 8, 14 * np.pi / 8, 15 * np.pi / 8, 2 * np.pi]:
    energies[thetas] = []
    for s in scales:
        sheetUpdate(thetas, s, 0.5 * s)
        energies[thetas].append([es.energy(), es.energy(es.EnergyType.Bending), es.energy(es.EnergyType.Membrane)])
    energies[thetas] = np.array(energies[thetas])

In [ ]:
import fractions
plt.figure(figsize=(12, 4))
for col, name in enumerate(['Full', 'Bending', 'Membrane']):
    plt.subplot(1, 3, 1 + col)
    for k, e in energies.items():
        f = fractions.Fraction(k / np.pi).limit_denominator(10)
        plt.plot(scales, e[:, col], label=r'$\theta = \frac{' + str(f.numerator) + '}{' + str(f.denominator) + '} \pi$')
        plt.title(f'{name} Energy')
    plt.grid()
    plt.legend()
plt.tight_layout()